In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import seaborn as sns

In [2]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

#office
bgtrain = pd.read_csv("../../Desktop/work/data science/project/bakery data/train.csv")
clients = pd.read_csv("../../Desktop/work/data science/project/bakery data/cliente_tabla.csv")
town_state = pd.read_csv("../../Desktop/work/data science/project/bakery data/town_state.csv")
product_name = pd.read_csv("../../Desktop/work/data science/project/bakery data/producto_tabla.csv")
bgtest = pd.read_csv("../../Desktop/work/data science/project/bakery data/test.csv")
sample_submission = pd.read_csv("../../Desktop/work/data science/project/bakery data/sample_submission.csv")

#submission must have 6999251 rows, i.e. the length of the test data set

In [154]:
len(bgtest)

6999251

In [3]:
bgtrain = bgtrain.rename(columns={'Semana': 'Week_num',
                                  'Agencia_ID': 'Sales_Depot_ID',
                                  'Canal_ID': 'Sales_Channel_ID',
                                  'Ruta_SAK': 'Route_ID',
                                  'Cliente_ID': 'Client_ID',
                                  'Venta_uni_hoy': 'Sales_unit_this_week',
                                  'Venta_hoy': 'Sales_this_week',
                                  'Dev_uni_proxima': 'Returns_unit_next_week',
                                  'Dev_proxima': 'Returns_next_week',
                                  'Demanda_uni_equil': 'Adjusted_demand',
                                  'Producto_ID': 'Product_ID'})

bgtest = bgtest.rename(columns={'Semana': 'Week_num',
                                  'Agencia_ID': 'Sales_Depot_ID',
                                  'Canal_ID': 'Sales_Channel_ID',
                                  'Ruta_SAK': 'Route_ID',
                                  'Cliente_ID': 'Client_ID',
                                  'Venta_uni_hoy': 'Sales_unit_this_week',
                                  'Venta_hoy': 'Sales_this_week',
                                  'Dev_uni_proxima': 'Returns_unit_next_week',
                                  'Dev_proxima': 'Returns_next_week',
                                  'Demanda_uni_equil': 'Adjusted_demand',
                                  'Producto_ID': 'Product_ID'})

The most naive guess is to use the same adjusted demand from week 9.  If the client or product is not present in week 9, set adjusted demand to 7.

In [7]:
#this reduces the entries from 75 million to 10 million

simple_bgtrain = pd.DataFrame({'Client_ID': bgtrain[bgtrain['Week_num'] == 9]['Client_ID'],
                               'Product_ID': bgtrain[bgtrain['Week_num'] == 9]['Product_ID'],
                               'Adjusted_demand': bgtrain[bgtrain['Week_num'] == 9]['Adjusted_demand']})

In [27]:
simple_bgtest = pd.DataFrame({'Client_ID': bgtest['Client_ID'],
                              'Product_ID': bgtest['Product_ID']})

In [8]:
df = pd.DataFrame({'id': np.arange(len(bgtest)),
                   'Demanda_uni_equil': 7*np.ones(len(bgtest)).astype(int)})

In [14]:
simple_bgtrain[(simple_bgtrain['Client_ID'] == bgtest['Client_ID'].iloc[2]) & (simple_bgtrain['Product_ID'] == bgtest['Product_ID'].iloc[2])]['Adjusted_demand']

71784635    2
Name: Adjusted_demand, dtype: int64

In [70]:
for i in range(2000000):
    if len(simple_bgtrain[(simple_bgtrain['Client_ID'] == simple_bgtest['Client_ID'].iloc[i]) & (simple_bgtrain['Product_ID'] == simple_bgtest['Product_ID'].iloc[i])]) != 0:
        df.loc[i, 'Demanda_uni_equil'] = simple_bgtrain[(simple_bgtrain['Client_ID'] == simple_bgtest['Client_ID'].iloc[i]) & (simple_bgtrain['Product_ID'] == simple_bgtest['Product_ID'].iloc[i])]['Adjusted_demand'].iloc[0]

In [ ]:
for i in np.arange(4000000,6999251,1):
    if len(simple_bgtrain[(simple_bgtrain['Client_ID'] == simple_bgtest['Client_ID'].iloc[i]) & (simple_bgtrain['Product_ID'] == simple_bgtest['Product_ID'].iloc[i])]) != 0:
        df.loc[i, 'Demanda_uni_equil'] = simple_bgtrain[(simple_bgtrain['Client_ID'] == simple_bgtest['Client_ID'].iloc[i]) & (simple_bgtrain['Product_ID'] == simple_bgtest['Product_ID'].iloc[i])]['Adjusted_demand'].iloc[0]

In [ ]:
df.to_csv('simple_prediction.csv', index=False)

In [78]:
df.loc[2000000-3:2000000+3]

,id,Demanda_uni_equil
1999997,1999997,4
1999998,1999998,19
1999999,1999999,6
2000000,2000000,7
2000001,2000001,7
2000002,2000002,7
2000003,2000003,7


In [74]:
for i in np.arange(1,10,2):
    print(i)

1
3
5
7
9


In [24]:
simple_bgtrain.head()

,Client_ID,Product_ID,Adjusted_demand
63771751,15766,1212,1
63771752,15766,1238,2
63771753,15766,1240,2
63771754,15766,1242,1
63771755,15766,1250,10


In [25]:
bgtest.head()

,id,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [26]:
bgtest[bgtest['Client_ID'] == 4639078]

,id,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID
0,0,11,4037,1,2209,4639078,35305
801547,801547,10,4037,1,2209,4639078,35305
1165993,1165993,11,4037,1,2209,4639078,35309
6120717,6120717,10,4037,1,2209,4639078,40930


In [172]:
bgtrain[(bgtrain['Client_ID'] == bgtest['Client_ID'].iloc[25]) & (bgtrain['Product_ID'] == bgtest['Product_ID'].iloc[25])]

,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID,Sales_unit_this_week,Sales_this_week,Returns_unit_next_week,Returns_next_week,Adjusted_demand
2199785,3,1227,1,2822,3342223,43285,6,31.68,0,0.0,6
13288004,4,1227,1,2822,3342223,43285,6,31.68,0,0.0,6
24138871,5,1227,1,2822,3342223,43285,8,42.24,0,0.0,8
34653220,6,1227,1,2822,3342223,43285,10,52.80,0,0.0,10
44939775,7,1227,1,2822,3342223,43285,7,36.96,0,0.0,7


In [158]:
bgtest.head()

,id,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [157]:
bgtest.tail()

,id,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID
6999246,6999246,11,2057,1,1153,4379638,1232
6999247,6999247,10,1334,1,2008,970421,43069
6999248,6999248,11,1622,1,2869,192749,30532
6999249,6999249,11,1636,1,4401,286071,35107
6999250,6999250,11,1625,1,1259,978760,1232


In [160]:
len(bgtrain[bgtrain['Client_ID'] == 0])

0

In [19]:
bgtrain.sample(5)

,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID,Sales_unit_this_week,Sales_this_week,Returns_unit_next_week,Returns_next_week,Adjusted_demand
50506279,7,2029,1,2126,4701793,43058,2,18.70,0,0.0,2
64789826,9,1137,1,1450,1812679,1250,12,91.68,0,0.0,12
57420304,8,1431,1,1233,4528376,43202,1,8.38,0,0.0,1
30422427,5,2055,1,2056,4409546,5380,7,51.87,0,0.0,7
74177036,9,25759,1,2815,4702357,36610,160,123.20,0,0.0,160


In [29]:
bgtest.sample(10)

,id,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID
1911828,1911828,10,1337,1,1240,1487018,1240
6611620,6611620,10,1219,1,2806,4672330,43285
4531082,4531082,10,2032,1,2153,288188,37361
4373919,4373919,10,1123,1,1236,1898437,2233
5754159,5754159,10,1385,4,6614,4383899,32937
2264072,2264072,11,1954,1,2838,121778,30532
493589,493589,10,23899,1,4103,443507,5621
5007804,5007804,10,1236,1,1202,683825,31423
1022657,1022657,11,2653,1,1042,406102,1146
5784453,5784453,10,1315,1,5055,100454,32861


In [57]:
print(len(bgtrain['Client_ID'].unique()))
print(len(bgtest['Client_ID'].unique()))

880604
745164


In [61]:
len(np.intersect1d(bgtrain['Client_ID'].unique(),bgtest['Client_ID'].unique()))

735501

Many clients have no sales in week 9.  Most of these clients only have sales in week 5.

In [ ]:
simple_bgtrain = pd.DataFrame({'Client_ID': bgtrain[bgtrain['Week_num'] == 9]['Client_ID'], 'Adjusted_demand': bgtrain[bgtrain['Week_num'] == 9]['Adjusted_demand']})

In [113]:
cid_train = np.sort(bgtrain['Client_ID'].unique())
cid_train_simple = np.sort(simple_bgtrain['Client_ID'].unique())
cid_test = np.sort(bgtest['Client_ID'].unique())
cid_test_not_train = np.sort(np.setdiff1d(cid_test,cid_train))
cid_test_not_train_simple = np.sort(np.setdiff1d(cid_test,cid_train_simple))
cid_all = np.sort(np.unique(np.concatenate((cid_train,cid_test))))

In [63]:
len(cid_test_not_train)

9663

In [92]:
cid_train

array([        26,         60,         65, ...,   19988629,   99999999,
       2015152015], dtype=int64)

In [93]:
np.max(bgtrain['Client_ID'].unique())

2015152015

In [165]:
df_submission = pd.DataFrame({'id': np.zeros(len(cid_all)).astype(int),
                              'Demanda_uni_equil': np.zeros(len(cid_all)).astype(int)})

df_submission.head()

,id,Demanda_uni_equil
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [169]:
df_submission.loc[1, 'Demanda_uni_equil'] = 8

In [170]:
df_submission.loc[0] = 1, 2
df_submission.head()

,id,Demanda_uni_equil
0,1,2
1,0,8
2,0,0
3,0,0
4,0,0


In [53]:
for i in range(100):
    if len(bgtrain[(bgtrain['Client_ID'] == df_submission['id'].iloc[i]) & (bgtrain['Week_num'] == 9)]['Adjusted_demand']) == 0:
        df_submission.loc[i, 'Demanda_uni_equil'] = 0
    
    else:
        df_submission.loc[i, 'Demanda_uni_equil'] = np.floor(bgtrain[(bgtrain['Client_ID'] == df_submission['id'].iloc[i]) & (bgtrain['Week_num'] == 9)]['Adjusted_demand'].mean()).astype(int)

In [96]:
for i in cid_train_simple:
    if i<16000:
        print(i)

26
60
65
101
105
106
107
262
465
772
786
791
795
797
906
1080
1266
1311
1436
1438
1589
2003
2006
2010
2011
2012
2015
2017
2018
2022
2023
2024
2025
2029
2033
2042
2046
2047
2049
2051
2052
2056
2058
2059
2060
2061
2062
2065
2066
2072
2073
2084
2093
2098
2100
2101
2102
2103
2104
2105
2107
2109
2111
2113
2117
2118
2119
2122
2123
2129
2136
2142
2143
2145
2147
2150
2152
2153
2154
2157
2161
2166
2169
2170
2177
2178
2179
2181
2182
2183
2184
2187
2195
2197
2198
2199
2200
2202
2203
2206
2207
2212
2213
2214
2219
2221
2224
2229
2232
2237
2241
2244
2257
2260
2261
2266
2269
2270
2271
2273
2274
2278
2279
2282
2284
2285
2287
2291
2293
2294
2295
2299
2300
2307
2310
2315
2318
2320
2321
2326
2330
2338
2346
2347
2354
2356
2358
2367
2372
2379
2382
2383
2386
2394
2396
2397
2399
2400
2401
2403
2404
2407
2408
2409
2410
2414
2425
2426
2429
2431
2433
2436
2437
2441
2442
2444
2445
2454
2457
2460
2463
2464
2465
2467
2468
2471
2472
2479
2482
2485
2486
2487
2488
2491
2494
2497
2498
2499
2500
2503
2506
2511
2512
251

In [97]:
for i in range(len(cid_train_simple)):
    df_submission.loc[i] = cid_train_simple[i], np.floor(simple_bgtrain[simple_bgtrain['Client_ID'] == cid_train_simple[i]]['Adjusted_demand'].mean()).astype(int)

In [100]:
df_submission.to_csv('test_submission.csv')

In [120]:
df_submission.iloc[len(cid_train_simple)-10:len(cid_train_simple)+1]

,id,Demanda_uni_equil
765565,10351784,13
765566,10351790,12
765567,10351796,14
765568,10351802,16
765569,10351808,12
765570,10351814,7
765571,11011586,18
765572,11693264,12
765573,19988629,5
765574,2015152015,8


In [112]:
df_submission.iloc[len(cid_train_simple)-1]

id                   2015152015
Demanda_uni_equil             8
Name: 765574, dtype: int32

In [118]:
avg_guess = np.round(df_submission.loc[0:len(cid_train_simple), 'Demanda_uni_equil'].mean()).astype(int)

avg_guess

6

In [119]:
for i in range(len(cid_test_not_train_simple)):
    df_submission.iloc[len(cid_train_simple)+i] = cid_test_not_train_simple[i], avg_guess

In [122]:
df_submission.iloc[len(cid_test_not_train_simple)+len(cid_train_simple)-2:len(cid_test_not_train_simple)+len(cid_train_simple)+2]

,id,Demanda_uni_equil
818250,9838323,6
818251,99999999,6
818252,0,0
818253,0,0


In [125]:
df_submission.iloc[len(cid_test_not_train_simple)+len(cid_train_simple)]

id                   0
Demanda_uni_equil    0
Name: 818252, dtype: int32

In [126]:
df_submission.drop(df_submission.index[df_submission.index >= len(cid_test_not_train_simple)+len(cid_train_simple)], inplace = True)

In [127]:
df_submission.tail()

,id,Demanda_uni_equil
818247,9747048,6
818248,9747716,6
818249,9780515,6
818250,9838323,6
818251,99999999,6


In [128]:
sample_submission.tail()

,id,Demanda_uni_equil
6999246,6999246,7
6999247,6999247,7
6999248,6999248,7
6999249,6999249,7
6999250,6999250,7


In [129]:
df_submission_copy = df_submission.copy()

In [143]:
#undo mistake
df_submission = df_submission_copy.copy()

In [145]:
df_submission = df_submission.sort_values(by = ['id'])

In [149]:
df_submission.tail()

,id,Demanda_uni_equil
765571,11011586,18
765572,11693264,12
765573,19988629,5
818251,99999999,6
765574,2015152015,8


In [151]:
df_submission.to_csv('simple_prediction.csv', index=False)